# Assoziaionsanlayse

Untersucht wird nur der Datensatz der Rotweine (~1600 Zeilen).
Der Datensatz der Weißweine ist zu groß (~5000 Zielen).

## Setup

In [ ]:
# Setup
from mlxtend.frequent_patterns import apriori, association_rules
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas.plotting

In [ ]:
# Einelsen der Daten
df_red = pd.read_csv('winequality-red-filtered.csv', sep=';', header=0) 
df_white = pd.read_csv('winequality-white-filtered.csv', sep=';', header=0) 

## Scatter-Plots

In [ ]:
def scatter(df, columns):
    pd.plotting.scatter_matrix(df[columns],
        figsize=(15, 15),
        marker="o",
        c=df['quality'].values,
        s=30,
        alpha=0.8,
    )
    plt.show()

### Feste Säureanteile - Citronensäure - PH-Wert - Dichte

In [ ]:
scatter(df_red, ["fixed acidity","citric acid","density","pH"])

### Schwefeloxide - freie Schwefeloxide

In [ ]:

scatter(df_red, ["free sulfur dioxide","total sulfur dioxide"])

### Flüchtige Säureanteile - Alkohol - Qualität

In [ ]:

scatter(df_red, ["volatile acidity","alcohol","quality"])

## Einteilung in Klassen
Um Support, Konfidenz und Lift verschiedener Ereignisse zu berechnen, werden die kontinuierlichen Daten diskretisiert.

In [ ]:
# Einteilung in je 3 Klassen (unter 25%, 25-75%, ueber 75% Quantil)
# ==> Binäre Matrix für mlextends Funktionen
df = pd.DataFrame()
for c in df_red.columns:
    q_25 = df_red[c].quantile(q=0.25)
    q_75 = df_red[c].quantile(q=0.75)
    print(f'{c}:  {q_25}  {q_75}')
    #df[f'{c}_low'] = df_red[c].le(q_25).astype(int)
    #df[f'{c}_mid'] = df_red[c].between(q_25, q_75, inclusive='right').astype(int)
    df[f'{c}_low'] = df_red[c].le(q_75).astype(int)
    df[f'{c}_high'] = df_red[c].gt(q_75).astype(int)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

### Konfidenz

In [ ]:
# hohe konfidenz
""" Ergebnis:
- zusammenhang niedriger freier schwefelanteil und niedriger gesamtschwefelanteil
- wenig alkohol -> niedrigere qualitaet (wenig alkhol, weniger suesse trauben, billiger)
- wenig sulfate -> niedrigere qualitaet
- viel schwefel -> niedrigere qualitaet (schwefel = haltbarkeitsmittel, mehr noetig bei faulen trauben)
- hohe fluechtige saeureanteile -> niedrigere qualitaet
- zusammenhang hohe citronensaeuere und viel feste saeure
- zusammenhang ph-wert saeure
- zusammenhang freier und gesamt schwefel
- zusammenhang hoher alkohol -> geringe dichte
==> zusammenhaenge bestaetigt
==> neu: kriterien fuer schlechten wein gefunden (lift allerdings oft nicht hoch)

2 Klassen:
- hohe dichte -> alkohol low und geringere qualitaet
- wenig citronensaeure -> qualiaet low (geringer lift)
- ph high -> qualitaet low (geringer lift)
"""
# Auch mit max_len=3 und head(30) getestet
analysis = apriori(df, min_support=0.05, use_colnames=True, max_len=2)
rules = association_rules(analysis, min_threshold=0.00)
rules.sort_values(by="confidence", ascending=False, inplace=True)
print(rules.head(20))

### Lift

In [ ]:
# hoher lift
"""Erkenntnis
- alkohol high -> quality high (geringe konfidenz aber hoher lift)
- fluechtige saeuren low -> quality_high (geringe konfidenz, aber hoher lift)
- viele sulfate -> quality high
==> nicht auschlaggebend, aber macht guten wein wahrscheinlicher, als 2./3. auswahlkriterium

2 Klassen:
- viel citronensaeure -> quality high (gerine konfidenz)
"""
analysis = apriori(df, min_support=0.05, use_colnames=True, max_len=2)
rules = association_rules(analysis, min_threshold=0.00)
rules.sort_values(by="lift", ascending=False, inplace=True)
print(rules.head(20))

In [ ]:
# lift near 1
""" Erkenntis
Leider Attribute findbar die keinen Einfluss auf gut/schlechte Qualitaet haben.
=> So notieren und zeigen, dass man an lift near 1 gedacht hat

2 Klassen (Mit max_len2):
hoher restzucker, wenig fluechtige, .... -> keinen Einfluss auf qualitaet low
"""
analysis = apriori(df, min_support=0.05, use_colnames=True, max_len=2)
rules = association_rules(analysis, min_threshold=0.00)
rules = rules[rules.lift.between(0.95,1.05)]
rules = rules[rules.consequents.apply(lambda c: bool(c.intersection({"quality_low","quality_mid","quality_high"})))]
rules.sort_values(by="lift", ascending=True, inplace=True)
print(rules)

In [ ]:
# lift low
""" Erkenntis
- wenig freie sauere -> unwahrscheinlicher dass wein schlecht
- viele sulfate -> unwahrscheinlicher dass wein schlecht
- mittlere dicht -> hoher alkohol gehalt unwahrscheinlicher

2 Klassen (und max_len=2)
- wenig alk -> guter wein unwahrscheinlicher
- und viel mehr
"""
analysis = apriori(df, min_support=0.05, use_colnames=True, max_len=2)
rules = association_rules(analysis, min_threshold=0.00)
rules = rules[rules.consequents.apply(lambda c: bool(c.intersection({"quality_low","quality_mid","quality_high"})))]
rules.sort_values(by="lift", ascending=True, inplace=True)
print(rules.head(30))